In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [2]:
from pypinyin import pinyin, lazy_pinyin, Style
# test pinyin
get_pinyin = lambda text: ' '.join([p[0] for p in pinyin(text, style=Style.TONE3)]).strip()
get_pinyin('今天天气怎么样')

'jin1 tian1 tian1 qi4 zen3 me yang4'

In [3]:
import argparse
import falcon
from hparams import hparams, hparams_debug_string
import os
from synthesizer import Synthesizer
synthesizer = Synthesizer()

In [5]:
import time
ckpt_step = 320000
checkpoint = r'logs-tacotron/model.ckpt-{}'.format(ckpt_step)

toc = time.time()
synthesizer.load(checkpoint)
print(time.time() - toc)

Constructing model: tacotron
Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead
Initialized Tacotron model. Dimensions: 
  embedding:               (1, ?, 256)
  prenet out:              (1, ?, 128)
  encoder out:             (1, ?, 256)
  decoder out (r frames):  (1, ?, 400)
  decoder out (1 frame):   (?, ?, 80)
  postnet out:             (?, ?, 256)
  linear out:              (?, ?, 1025)
  stop token:              (?, ?)
Loading checkpoint: logs-tacotron/model.ckpt-320000
INFO:tensorflow:Restoring parameters from logs-tacotron/model.ckpt-320000
20.14705729484558


In [6]:
import soundfile as sf
import io
import numpy as np
text = '''
中美贸易战休战关键点, 九十天谈判无果，关税仍将提高. 中美贸易战暂时休战, 符合外界此前的普遍预期
'''
text='''
预计 二零一六 年将完成运输总周转量 九百六十九 亿吨、旅客运输量 四八七七六 万人次、货邮运输量 六百六十九点九 万吨，同比分别增长, 百分之十一点八 和 百分之六点零。
'''

text='''
蒙眼”挑战引热潮危险恶搞影片YouTube下禁令. 惊悚片,《蒙上你的眼》,的恐怖元素让观众深深着迷，纷纷拍摄,《蒙眼》,挑战视频，甚至蒙眼开车发生了意外。
'''

# text='''
# 美爱海普右
# '''

text='''
省级文保单位满院溜达鸡,一丘之貉，一个苹果
'''



text_input = get_pinyin(text)

# filter control character 
for s in set(list('？。！，.、')):
    text_input = text_input.replace(s, ',')

# split
text_input = [t.strip() + ' ' for t in text_input.split(',') if len(t.strip()) > 1]

text_input = ['十七日上午, 习近平来到天津考察调研', '在南开大学, 他参观了百年校史主题展览', '与部分院士专家和中青年师生代表互动交流', '察看了化学学院和元素有机化学国家重点实验室', 
              '详细了解南开大学历史沿革, 学科建设, 人才队伍, 科研创新等情况']
text_input = [get_pinyin(t) for t in text_input]
# text_input = ["wei1 xin4 zhi1 fu4 zhang1 xiao3 long2 han3 jian4 lou4 mian4"]



# # synthesize
datas = []
for t in text_input:
    audio_bytes= synthesizer.synthesize(t)
    audio_io = io.BytesIO()
    audio_io.write(audio_bytes)
    audio_io.seek(0)
    s, fs = sf.read(audio_io)
    s /= np.max(np.abs(s))
    datas.append(np.array(s))
data = np.hstack(datas)
data_io = io.BytesIO()
sf.write(data_io, data, fs, format='WAV')

# # # text_input = pinyin_no_space('中美贸易战休战关键点, 九十天谈判无果，关税仍将提高. 中美贸易战暂时休战, 符合外界此前的普遍预期。 ')
# for s in list(r'，。！？：；“”'):
#     text_input = text_input.replace(' ' + s, s)

# data = synthesizer.synthesize(text_input) 
text_input

['shi2 qi1 ri4 shang4 wu3, xi2 jin4 ping2 lai2 dao4 tian1 jin1 kao3 cha2 diao4 yan2',
 'zai4 nan2 kai1 da4 xue2, ta1 can1 guan1 le5 bai3 nian2 xiao4 shi3 zhu3 ti2 zhan3 lan3',
 'yu3 bu4 fen4 yuan4 shi4 zhuan1 jia1 he2 zhong1 qing1 nian2 shi1 sheng1 dai4 biao3 hu4 dong4 jiao1 liu2',
 'cha2 kan4 le5 hua4 xue2 xue2 yuan4 he2 yuan2 su4 you3 ji1 hua4 xue2 guo2 jia1 zhong4 dian3 shi2 yan4 shi4',
 'xiang2 xi4 liao3 jie3 nan2 kai1 da4 xue2 li4 shi3 yan2 ge2, xue2 ke1 jian4 she4, ren2 cai2 dui4 wu3, ke1 yan2 chuang4 xin1 deng3 qing2 kuang4']

In [7]:
import IPython.display as ipd
ipd.Audio(data_io.getvalue())